In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, upper, translate, when, regexp_replace, trim, lit, create_map, to_timestamp, date_format
)
from pyspark.sql.types import IntegerType, StringType
from itertools import chain
import unicodedata

# =======================================================================
# 1. CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# =======================================================================
conf = SparkConf()
conf.set(
    "spark.jars.packages",
    "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901"
)
conf.set(
    "spark.hadoop.fs.s3a.aws.credentials.provider",
    "com.amazonaws.auth.InstanceProfileCredentialsProvider"
)

spark = SparkSession.builder.appName("DataCleaningAtendimentosUPA").config(conf=conf).getOrCreate()

# =======================================================================
# 2. FUNÇÕES DE LIMPEZA E PADRONIZAÇÃO
# =======================================================================

# Função auxiliar para padronizar nomes de colunas no Python
def padronizar_coluna_py(col_name):
    """
    Padroniza o nome da coluna: remove acentos, mantém alfanuméricos/espaços,
    converte para maiúsculo e substitui espaços por '_'.
    """
    # Normalização Unicode (melhor forma de remover acentos no Python)
    normalized_name = ''.join(
        c for c in unicodedata.normalize('NFKD', col_name)
        if not unicodedata.combining(c)
    )
    # Remove caracteres especiais que não sejam letras, números ou espaço
    cleaned_name = ''.join(c for c in normalized_name if c.isalnum() or c == ' ')
    
    # Converte para maiúsculo e substitui espaços por _
    final_name = cleaned_name.upper().replace(' ', '_')
    return final_name

# Função para limpar e padronizar o CONTEÚDO de colunas string no PySpark
# Simula a lógica NFKD + ascii ignore + regex replace
def limpar_coluna_string_pyspark(col_name):
    """
    Aplica a limpeza do conteúdo da coluna: maiúsculas, remoção de acentos comuns
    e remoção de caracteres especiais, deixando apenas letras, números e espaços.
    """
    col_expr = upper(col(col_name))
    
    # Remoção de acentos mais comuns usando translate
    acentos = "ÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ"
    sem_acentos = "AAAAAEEEEIIIIOOOOOUUUUC"
    col_expr = translate(col_expr, acentos, sem_acentos)
    
    # Remove caracteres que não são letras (A-Z), números (0-9) ou espaço
    col_expr = regexp_replace(col_expr, r"[^A-Z0-9 ]", "")
    
    # Remove espaços extras no início e fim
    col_expr = trim(col_expr)
    
    return col_expr

# =======================================================================
# 3. DEFINIÇÕES DE CAMINHO
# =======================================================================
INPUT_PATH = "s3a://bucket-raw-upa-connect-sofh/basesExternas/Atendimentos/2025-06-06_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv"

# Caminhos de destino
FINAL_OUTPUT_DIR = "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos"
FINAL_FILENAME = "AtendimentosTratada.csv"
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging"


# =======================================================================
# 4. LEITURA DOS DADOS
# =======================================================================
print("Lendo dados do S3...")
# Assumindo o encoding ISO-8859-1 (latin1) para a leitura inicial, 
# pois é comum em dados brasileiros e o script pandas sugeria isso.
df_raw = spark.read.option('delimiter', ';') \
                     .option('header', 'true') \
                     .option('nullValue', 'null') \
                     .option('encoding', 'ISO-8859-1') \
    .csv(INPUT_PATH)

# =======================================================================
# 5. SELEÇÃO E FILTRO INICIAL
# =======================================================================
colunas_desejadas = ["Data do Atendimento", "Sexo", "Tipo de Unidade", "Descrição do Procedimento", "Descrição do CBO", "Nacionalidade"]
df_limpo = df_raw.select(*colunas_desejadas)

# Filtra apenas Tipo de Unidade = UPA
df_final = df_limpo.filter(col("Tipo de Unidade") == "UPA")

# =======================================================================
# 6. LIMPEZA E TRANSFORMAÇÃO DE COLUNAS
# =======================================================================

# 6.1. Limpeza de colunas string (Procedimento, Nacionalidade, CBO)
colunas_para_limpar = ["Descrição do Procedimento", "Nacionalidade", "Descrição do CBO"]

for c in colunas_para_limpar:
    df_final = df_final.withColumn(c, limpar_coluna_string_pyspark(c))

# 6.2. Mapeamento da Categoria do Procedimento
mapa_categoria_dict = {
    'ATENDIMENTO MEDICO EM UNIDADE DE PRONTO ATENDIMENTO': 'PRONTO ATENDIMENTO',
    'ATENDIMENTO DE URGENCIA C OBSERVACAO ATE 24 HORAS EM ATENCAO ESPECIALIZADA': 'URGENCIA/OBSERVACAO',
    'ACOLHIMENTO COM CLASSIFICACAO DE RISCO': 'ACOLHIMENTO'
}

# Cria um mapa PySpark para usar com a função 'when' ou 'select'
mapping_expr = create_map([lit(x) for x in chain(*mapa_categoria_dict.items())])

df_final = df_final.withColumn(
    'Tipo Procedimento',
    mapping_expr.getItem(col('Descrição do Procedimento'))
)

# 6.3. Conversão da Data para ISO 8601
# Tentativa de inferir o formato da data original (pode ser "DD/MM/YYYY HH:MM:SS" ou similar)
# Para datas mais complexas, usamos to_timestamp
df_final = df_final.withColumn(
    "Data Iso",
    date_format(
        to_timestamp(col("Data do Atendimento"), "dd/MM/yyyy HH:mm:ss"), # Formato comum para CSV brasileiro
        "yyyy-MM-dd'T'HH:mm:ss"
    )
)

# 6.4. Drop da coluna original de data
df_final = df_final.drop("Data do Atendimento")

# =======================================================================
# 7. PADRONIZAÇÃO DOS NOMES DAS COLUNAS
# =======================================================================

# 7.1. Obtém os nomes atuais
current_columns = df_final.columns

# 7.2. Aplica a função de padronização aos nomes
new_columns = [padronizar_coluna_py(c) for c in current_columns]

# 7.3. Renomeia as colunas no DataFrame
df_final = df_final.toDF(*new_columns)
print("\nSchema Final Padronizado:")
df_final.printSchema()


# =======================================================================
# 8. SALVANDO E RENOMEANDO O RESULTADO NO S3
# =======================================================================

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")

# Usamos o encoding 'UTF-8' na saída para o trusted, que é o padrão moderno
df_final.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'UTF-8') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR), True) 
        
        print(f"\n✅ Dados salvos e renomeados com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# Encerra a sessão Spark
spark.stop()


:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-680acac1-d96b-4ff1-b60f-28f25f108316;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 362ms :: artifacts dl 18ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Lendo dados do S3...


25/10/17 13:32:07 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
/usr/local/lib/python3.7/site-packages/pyspark/sql/column.py:462: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  FutureWarning,



Schema Final Padronizado:
root
 |-- SEXO: string (nullable = true)
 |-- TIPO_DE_UNIDADE: string (nullable = true)
 |-- DESCRICAO_DO_PROCEDIMENTO: string (nullable = true)
 |-- DESCRICAO_DO_CBO: string (nullable = true)
 |-- NACIONALIDADE: string (nullable = true)
 |-- TIPO_PROCEDIMENTO: string (nullable = true)
 |-- DATA_ISO: string (nullable = true)


Escrevendo dados temporariamente em: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos/_temp_staging


25/10/17 13:32:19 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/17 13:32:20 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Dados salvos e renomeados com sucesso para: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos/AtendimentosTratada.csv
